![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Mathematics/InterpretingStatisticalData/interpreting-statistical-data.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

<h1 align='center'>Developing Statistical Reasoning: Interpreting Statistical Data</h1>

## Introduction

Statistical reasoning plays an increasingly large role in everyday's life, from politics (opinion polls) to sports (statistics about sports teams), through many news about health or economy for example. The goal of this notebook is to introduce and illustrate two important notions about interpreting statistical data: the notions of confidence *interval* and confidence *level*.

This notebook is split into two parts. The first part is a short lecture about the statistical concepts we want to work with. The second one provides some interactive tools to experiment with the concepts and illustrates their use in some real-world applications.

## Part 1 - Statistical Concepts

### 1A. Mean and Standard Deviation

We first introduce well known concepts: mean and standard deviation.

<div class="alert alert-warning">
    <font color="black"><i>Definition:</i> <b>Mean.</b> Given a data set of $n$ numbers $\{x_1,x_2,\dots,x_n\}$, we denote the $\textbf{mean}$ of the data set $\overline{x}$ (pronounced "x bar") to be
$$ $$
$$\overline{x} = \frac{x_1+x_2+\cdots+x_n}{n}.$$</font>
</div>

**Example:** Suppose we have the following set of data: $\{13, 19, 7, 3\}$. Here, we have $4$ numbers, so $n=4$. The mean of this data set is given by:

$$\begin{align} 
\overline{x} &= \frac{x_1+x_2+x_3+x_4}{4} \\
&= \frac{13+19+7+3}{4} \\
&= 10.5 
\end{align}$$

<div class="alert alert-warning">
    <font color="black"><i>Definition:</i> <b>Standard Deviation.</b> Given a data set of $n$ numbers $\{x_1,x_2,\dots,x_n\}$, we define the $\textbf{standard deviation}$ of the data set, that we denote by $s$, as follows:
$$\displaystyle s = \sqrt{ \frac{1}{n}\left( (x_1-\overline{x})^2 + (x_2-\overline{x})^2 + \cdots + (x_n-\overline{x})^2 \right) }.$$</font>
</div>

**Example:** Suppose again that we have the following set of data: $\{13, 19, 7, 3\}$. In the previous example, we've calculated $n=4$ and $\overline{x} = 10.5$. The standard deviation of this data set is given to be:

$$\begin{align} s &= \sqrt{ \frac{1}{4}\left( (x_1-\overline{x})^2 + (x_2-\overline{x})^2 + (x_3-\overline{x})^2 + + (x_4-\overline{x})^2 \right) }  \\
&= \sqrt{ \frac{1}{4}\left( (13 - 10.5)^2 + (19-10.5)^2 + (7-10.5)^2 + (3-10.5)^2 \right) } \\
&= \sqrt{ \frac{1}{4}\left( 6.25 + 72.5 + 12.25 + 56.25 \right) } \\
&= \sqrt{ \frac{147.25}{4} } \\
&\approx 6.06733 \\
\end{align}$$

**Significance:** 
The standard deviation complements the mean by telling us how much we can expect that an element of our data set deviates from the mean. The mean alone does not differentiate between the case where all elements of our data set have the same value (for example $\{2,2,2,2\}$ with $\overline{x} = 2$ and $s=0$) or have value alternating around the mean ($\{1,3,1,3\}$ also has $\overline{x} = 2$ but $s=1$).

The standard deviation of a data set is a measure that quantifies the amount of variation in the data set. A lower standard deviation indicates that the values in the data generally lay closer to the mean of the data set, whereas a high standard deviation says that the data points are more spread apart and take over a wider range of values that are further from the mean.

### 1B. Sampling Data

We consider now the following context, inspired by real-world application of statistics. We want to obtain the average (mean) value of a statistic over a very large number of cases. For example, we could be interested in the average height of people in Canada. In order to obtain the *exact answer*, we would need to measure the height of all people living in Canada. This is obviously impossible. So we need to proceed in a different way: we will select a **random sample** of the Canadian population and obtain the height measurements of the people in that sample. And the question that naturally occurs is: how can we deduce anything about the average height of the whole population from this sample? Surprisingly, we will see that we can say something, using the notions of **confidence level**, **confidence interval** and **margin of error**, concepts you might have heard about especially in opinion polls. This is based on non-trivial statistics, and we will not explore the technical aspects, but illustrate these notions with concrete examples.

#### Confidence interval and confidence level
So there exists a quantity $x$ we want to estimate because we can not obtain it exactly (the average height of people living in Canada in our example). We have measured the height of a random sample of $n$ people, let's denote it by  $\{x_1,x_2,\dots,x_n\}$.

The first important point related to sampling data is to estimate the mean of a quantity, that is, we will not compute a single number to approximate $x$, but an **interval** $(x_\text{low},x_\text{high})$. So we decide to lose *precision*, as an interval is less precise than a single number. The key point is that we want to be **confident**  that the value $x$ is in that interval. This notion of confidence is important and we want to quantify it in some way. We would like to be able to say something like "I am $95\%$ confident that the value $x$ is in the interval $(x_\text{low},x_\text{high})$ I computed". This is a sentence we read often with opinion polls, aimed at describing their **accuracy**.

Now, for us, we want to understand how to interpret the expression "$95\%$ confident". In our example about averaging the height of Canadian people, it can be understood as follows: if we were going to repeat *often* (ideally an infinite number of times ... obviously this is impossible) and *independently* (our samples are truly random) the process of:
* selecting a random set of $n$ people
* measuring the height of the sampled people, $\{x_1,x_2,\dots,x_n\}$
* using the same method to deduce from $\{x_1,x_2,\dots,x_n\}$ the interval $(x_\text{low},x_\text{high})$

then $95\%$ of the intervals generated as above would contain the exact value $x$ we want to estimate.

So we have **no guarantee** about the precise interval $(x_\text{low},x_\text{high})$ we computed from a specific sample $\{x_1,x_2,\dots,x_n\}$ but we expect it contains $x$ with probability $95\%$. Note that we selected a confidence level that gives us an interval of 95%, as it's by far the most common number, but realistically we could set that level at whatever we wanted. Setting a higher confidence level results in a wider confidence interval (why?), whereas a lower confidence level results in a narrower confidence interval.

#### Computing a confidence interval for a given confidence level

The question now is: if we want to have a given confidence level (so we chose it), what is the method to compute $(x_\text{low},x_\text{high})$ from $\{x_1,x_2,\dots,x_n\}$? Let's denote the confidence level by $\beta$, a real number between $0$ and $1$ ($\beta=0.95$ means a confidence level of $95\%$).

The method to compute $(x_\text{low},x_\text{high})$ is as follows:
* Step 1. We calculate the mean $\overline{x}$ of the sampled set $\{x_1,x_2,\dots,x_n\}$, and its standard deviation $s$.
* Step 2. We find the **z-score** corresponding to the chosen confidence level $\beta$ (we will not discuss in this notebook how these z-scores are obtained, but they can be read in a table, see http://www.statisticshowto.com/tables/z-table/ and we provide a separate notebook to compute it given $\beta$).
* Step 3. Calculate $\displaystyle x_\text{low}=\overline{x}- z\frac{s}{\sqrt{n}}$ and $\displaystyle x_\text{high}= \overline{x}+ z\frac{s}{\sqrt{n}}$. 
So our confidence interval is:
$$(x_\text{low},x_\text{high}) = \displaystyle \left(\overline{x}- z\frac{s}{\sqrt{n}} ,\overline{x}+ z\frac{s}{\sqrt{n}} \right).$$

**Comments:** <br>
This formula is actually quite intuitive. We can see that this confidence interval is centred around the mean of the sampled data ($\overline{x}$), with some slack on both sides that depends on the standard deviation, the number of samples and the mysterious z-score. So intuitively, we assume that $\overline{x}$ is more or less our approximation of $x$; this makes sense. But if the standard deviation is large (i.e. we see lots of variation in the sampled measurements) then we widen the interval. Conversely, if we sample many data points (i.e. if $n$ is large) then we shrink the interval (because we divide by $\sqrt{n}$); again this make sense, as the more we sample the more we expect to get a good idea of the quantity we want to approximate. What is nice is that all these ideas combine into a quite simple formula, thanks to the z-score.

**Example:** <br> 
Engineers are concerned with designing devices operated by workers to achieve high productivity and comfort. The paper [Studies on Ergonomically Designed Alphanumeric Keyboards (1985)](https://pubmed.ncbi.nlm.nih.gov/4018810/) reports a study of preferred height for a keyboard. A sample of $n=31$ is given and a preferred keyboard height was reported by each of the $31$ sampled typists, with the sample average preferred being $\overline{x} = 80.0$ cm and the standard deviation being $s=2.0$ cm. Using the steps above, how may we obtain a 95% confidence interval for the real mean - the real preferred height for the population of experienced typists.

* Step 1. We know $n=31, \overline{x}=80$, and $s=2$.
* Step 2. Since we want a confidence level of 95%, we obtain an associated z-score of $1.96$. 
* Step 3. Calculating the estimates, we have: $\overline{x} \pm 1.96\cdot \frac{s}{\sqrt{n}} = 80 \pm (1.96)\frac{2.0}{\sqrt{31}} = 80.0 \pm 0.7 $. So $x_\text{low} = 79.3$ and $x_\text{high} = 80.7$. 

So our confidence interval is $(79.3,80.7)$ and the confidence level of this interval (the probability it contains the mean preferred height for the population of experienced typists) is 95%. we can observe that the interval is quite narrow, thanks to the small standard deviation and the large value of $n$.

#### Margin of error.
Last we introduce the notion of margin of error:

<div class="alert alert-warning">
    <font color="black"><i>Definition:</i> <b>Margin of Error.</b> In a confidence interval, we define the value $z\frac{s}{\sqrt{n}}$ to be the $\textbf{margin of error}$ of the confidence interval.</font>
</div>

So we can see that actually the margin of error is nothing else that the half-size of the interval, based on the standard deviation, the sample size and the z-score associated to the confidence level. 

This is exactly how you should understand the notion of margin of error that we see in many opinion polls: they have polled a sample of $n$ people, chosen a confidence level, computed the corresponding confidence interval and report the centre value $\overline{x}$ and the half-size of the interval as margin of error.

## Part 2 - Interactive Activities and Real Examples

### How Does the Sample Size, Confidence Level, and Margin of Error Affect Confidence Intervals?

When calculating confidence interval, margin of error, and determining what confidence level to use, the size of the random sample we use is important. 

In the example we develop now, we look at the height distribution of men, samples from an unknown population size (with unknown parameters). In an interactive tool below, we will be looking at the impact of the random sample size on the size of the confidence interval and the margin of error.

Using a 95% confidence level means that if we record the heights of say 1,000 men from the population over and over again and calculate the confidence interval, we expect about 95% of the confidence intervals will obtain the true average height of men in the population.

**Try it for yourself!**

The interactive tool below generates a plot with the ability to select the parameters of the distribution of the data, as well as the confidence interval for which you hope to find the mean. The red intervals contain the mean, within a confidence interval equal to that selected with the slider. With this tool you can appreciate visually how increasing the number of samples impacts positively the likelihood that the confidence intervals actually contain the mean height. It also shows the effect of having datasets with larger (and smaller) values for mean, and standard deviation.

In [ ]:
import scipy.stats as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets.widgets import IntSlider, FloatSlider, VBox, HBox, Label

sampleSize = IntSlider(value=100, min=10, max=2000, step=10)
conLev = FloatSlider(value=0.95, min=0.01, max=0.99, step=0.01)
sampleMean = IntSlider(value=175, min=150, max=200, step=1)
stDev = IntSlider(value=5, min=1, max=10, step=1)

fig1 = go.FigureWidget()
fig1.add_histogram(xbins=dict(size=2.5))

def response(change):
    with fig1.batch_update():
        np.random.seed(1)
        fig1.layout={}
        moe = max((st.norm.ppf(1-(1-conLev.value)/2) * (stDev.value / np.sqrt(sampleSize.value)), 0)) # Calculate z-score, then multiply by s/sqrt(n)
        barData = np.random.normal(sampleMean.value, stDev.value, sampleSize.value)       
        fig1.data[0].x = barData
        fig1.update_layout(xaxis_range=[sampleMean.value-50,sampleMean.value+50],
                          title=rf'Height Distribution of Men (n={sampleSize.value})',
                          xaxis_title='Height (cm)',
                          yaxis_title='Number of people',
                          xaxis = dict(showgrid=True))
        fig1.add_shape(type="rect", yref='paper',
                       x0=sampleMean.value - moe, y0=0, x1=sampleMean.value + moe, y1=0.99,
                       line=dict(color="red",width=2), fillcolor='red', opacity=0.4)
        fig1.add_annotation(x=sampleMean.value, y=1.0, yref='paper', text=f'{round(conLev.value*100,1)}% confidence interval')
    
sampleSize.observe(response, names="value")
conLev.observe(response, names="value")
sampleMean.observe(response, names="value")
stDev.observe(response, names="value")
response('')

VBox([fig1, 
    HBox([Label('Sample size'), sampleSize]), 
    HBox([Label('Confidence level'), conLev]), 
    HBox([Label('Mean'), sampleMean]), 
    HBox([Label('Standard Deviation'), stDev])])

In the interactive tool above, we've seen that increasing the sample size and lowering our confidence level leads to a narrower confidence interval within which we would expect to find the mean. When we generate data with a larger standard deviation, it has a wider range of potential values, so it *increases* the confidence interval that we would expect to find. Lastly, note how the mean itself doesn't affect the confidence interval, other than to change the point estimate that it's centered on

This is intuitive, as with more data, we can be more confident in our results. Similarly, we can make more precise estimates of the true mean if we're okay to accept that our guesses will be less confident, or to look at it the other way, we can 'cast a wider net' to capture the mean, which we can be more confident in being successful.

## Confidence Intervals and Confidence Levels in Media

In 2017, [The Hockey News](https://thehockeynews.com/news/article/front-office-confidence-rankings-part-2-fans-weigh-in-on-how-each-team-is-doing) created a survey asking people to mark each NHL team's front office on a scale from 1 to 5 in six different categories: roster building, cap management, draft and develop, trading, free agency, and their overall vision. Over 1300 people contributed in the survey and since not everyone was familiar with every team's management group, each NHL team had roughly 800 to 1000 votes. 

The goal was to **rank** the NHL teams in each of the 6 categories. For a given category, this would have been easy if each team had received as many marks as there were people contributing to the survey: it would have been sufficient to rank according to the mean mark. But as noted above, not every participant marked every team, so The Hockey news faced an issue akin to sampling marks, where a participant not marking a team in a category was considered as this (missing) mark being un-sampled. So instead of a mean mark per category for each team, The Hockey News computed a 95% confidence interval. That is, for each of the 30 teams, they  obtained six confidence intervals (one for each ranking category), with 95% certainty that the intervals contains the actual mark value. Now, each of these confidence intervals have their associated margin of error (there are 180 intervals in total) which averaged out to be 0.06, which is quite small.

Take for example the Nashville Predators, who ranked first overall out of the 30 teams (at the time).

<img src="./images/nashville-predators.jpg" alt="Drawing" style="height: 300px" style="width: 300px;"/>

Since the margin of error is about 0.06, the true mark (out of 5) of the team's ability in roster building likely (with probability 95%) lies in the confidence interval:

$$(4.4 - 0.06, 4.4 + 0.06) = (4.34, 4.46)$$

## Confidence Intervals and Margin of Error in Media

[Boston University of Public Health](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_Confidence_Intervals/BS704_Confidence_Intervals_print.html) analyzed data of 3,539 participants who attended the 7th Examination of the Offspring Cohort in The Framingham Heart Study. For people participating to the study, several health measures were taken, again with a missing data issue as not all participants submitted their information for each category, an issue similar to the hockey example above. The summarized data set is presented below.  

In [ ]:
# Create nested column labels 
inner_columns = ['Men, Sample Size','Men, Mean','Men, Standard Deviation','Women, Sample Size','Women, Mean','Women, Standard Deviation']

# Row names
indices_labels = ['Systolic Blood Pressure','Diastolic Blood Pressure','Total Serum Cholesterol', 'Weight', 'Height', 'Body Mass Index']
dataframe = pd.DataFrame( [ [1623, 128.2, 17.5, 1911,126.5, 20.1] , [1622, 75.6, 9.8, 1910, 72.6, 9.7] , [1544, 192.4, 35.2, 1766, 207.1, 36.7], [1612, 194.0, 33.8, 1984, 157.7, 34.6], [1545, 68.9, 2.7, 1781, 63.4, 2.5], [1545, 28.8, 4.6, 1781, 27.6, 5.9] ] , columns = inner_columns)

# Group the labels as split by " , " 
labels = dataframe.columns.str.split(', ', expand=True).values
dataframe.columns = pd.MultiIndex.from_tuples([x for x in labels])
original_index_list = dataframe.index.tolist()

for current_index in range(len(original_index_list)):
    idx = original_index_list.index(current_index)
    original_index_list[current_index] = indices_labels[current_index]
    
d = dict(selector="th", props=[('text-align', 'left')])
dataframe.index = original_index_list

display(dataframe.style.set_properties(**{'width':'12em', 'text-align':'center'}).set_table_styles([d]))

With 95% confidence, the confidence interval of Men's Systolic Blood Pressure is $\left(127.35, 129.05\right)$ and the margin of error is $0.85$. That is, there is a 95% chance that the actual average of Men's Systolic Blood Pressure in the population belongs in this confidence interval.

## Analyzing Statistical Data Presented in the Media

According to an Opinion Article in the National Post [Ontario Teachers Are Overpaid](https://nationalpost.com/opinion/allan-richarz-ontarios-teachers-are-overpaid), the article states that "Ontario’s Sunshine List of public-sector employees making over \\$100,000 each year. Thousands of elementary and secondary school teachers, admin and staff make the list; with some teachers reaching as high as \\$133,000 annually" and the article argues that "teachers in the province are overpaid; the work is not that onerous nor specialized and the hours not too taxing."

<img src="./images/ontario-teacher-salary.png" alt="Drawing" style="width: 500px;"/>

However, looking at the high school teacher salary in Ontario as reported on [PayScale](https://www.payscale.com/research/CA/Job=High_School_Teacher/Salary/004c9fd6/Toronto-ON), shows that these teacher may not necessarily be overpaid.

It matters first as to which group of workers we are looking at in the industry for instance, some employees such as school administrators make more than the average secondary school teachers and can drag the average salary up.

It also matters to look at how long they have been working in the industry. A count of 61 entry level teachers (working for 0-5 years) reported an income of \\$52,000 a year and a count of 44 mid-career level teachers (working for 5-10 years) reported an income of \\$56,000 a year.

It is not until we get to the count of 33 experienced teachers (working for 10-15 years) reporting an income of \\$75,000 a year and late career teachers (working for 15-20 years) reporting an income of \\$88,000 that we see comparable incomes as reported by the article.

Unfortunately we cannot obtain a confidence interval, as the individual data values in the data set are not provided by PayScale.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)